In [47]:
import numpy as np
import pandas as pd
torch.manual_seed(42)
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.model_selection import train_test_split

In [48]:
# Load original dataset
df = pd.read_csv("bending_machine_data.csv")  # Replace with actual dataset path
target_col = "result"  # Define the target column

In [50]:
# Prepare data for training
X = df.drop(columns=[target_col]).values
y = df[target_col].values
input_dim = X.shape[1]

In [51]:
# Define Diffusion Model
class DiffusionModel(nn.Module):
    def __init__(self, input_dim, timesteps=1000):
        super(DiffusionModel, self).__init__()
        self.timesteps = timesteps
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x, t):
        return self.model(x)

In [52]:
# Training setup
def train_diffusion(epochs=500, batch_size=32):
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    diffusion_model = DiffusionModel(input_dim)
    optimizer = optim.Adam(diffusion_model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        for real_data in dataloader:
            real_data = real_data[0]
            optimizer.zero_grad()
            noisy_data = real_data + torch.randn_like(real_data) * 0.1  # Adding noise
            recon_data = diffusion_model(noisy_data, None)
            loss = loss_fn(recon_data, real_data)
            loss.backward()
            optimizer.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch}: Loss {loss.item():.4f}")

train_diffusion()

Epoch 0: Loss 515.7001
Epoch 50: Loss 0.2963
Epoch 100: Loss 0.1353
Epoch 150: Loss 0.0762
Epoch 200: Loss 0.0366
Epoch 250: Loss 0.0187
Epoch 300: Loss 0.0177
Epoch 350: Loss 0.0117
Epoch 400: Loss 0.0179
Epoch 450: Loss 0.0117


In [53]:
# Generate synthetic data
num_samples = 500
z = torch.randn(num_samples, input_dim)
diffusion_model = DiffusionModel(input_dim)
synthetic_data = diffusion_model(z, None).detach().numpy()

In [54]:
# Save synthetic data
synthetic_df = pd.DataFrame(synthetic_data, columns=df.drop(columns=[target_col]).columns)
synthetic_df[target_col] = np.random.uniform(y.min(), y.max(), size=synthetic_df.shape[0])
synthetic_df.to_csv("dm_synthetic_data.csv", index=False)

In [55]:
# Create augmented dataset
augmented_df = pd.concat([df, synthetic_df], ignore_index=True)
augmented_df.to_csv("dm_augmented_data.csv", index=False)